In [ ]:
import os
import sys
import json
import re
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from pandas import DataFrame
import numpy as np
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras_self_attention import SeqSelfAttention
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from keras.layers.recurrent import GRU
from keras.models import load_model
from keras.layers import TimeDistributed, Input, Layer
from tensorflow.keras.layers import Conv2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential,Model,Input
from keras.layers import Bidirectional
from keras.layers import concatenate
import keras
from keras import initializers
from tcn import TCN, tcn_full_summary
from keras import backend as K
import math
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
#from transformers import BertTokenizer, TFBertModel, BertConfig

np.set_printoptions(threshold=np.inf)
import keras
#nltk.download('stopwords')
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
#embed = TFBertModel.from_pretrained("bert-base-uncased")
#stop_words = set(stopwords.words('english'))

In [ ]:
MAX_LENGTH = 50
EMBEDDING_DIMENSION = 512
WINDOWS = 5
BATCH_SIZE = 32

In [ ]:
def preprocessing(company, date):
    str = company + '/' + date
    filepath = os.getcwd() + '/tweet/preprocessed/' + str
    list = []
    with open(filepath, 'r') as file:
        for line in file:
            data = json.loads(line)
            list.append(data['text'])
    return list

In [ ]:
X = pd.read_csv(os.getcwd()+'/price/alltrainings.csv')
X_test = pd.read_csv(os.getcwd()+'/price/alltestings.csv')
X_val = pd.read_csv(os.getcwd()+'/price/allvalidations.csv')
y = X['label']
y_test = X_test['label']
y_val = X_val['label']
y = y[WINDOWS:]
y_val = y_val[WINDOWS:]
y_test = y_test[WINDOWS:]

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    #tweet = tweet.replace("#", "").replace("_", "").replace("$", "") #Remove hashtag sign but keep the text
    tweet = tweet.replace("rt", "").replace("AT _ USER", "").replace("URL", "")
    words = tweet.split()
    wordsnew = tweet.split()
    for i in range(len(words)):       
        if '$' == words[i]:
         
            wordsnew = wordsnew[:i+1] + wordsnew[i + 2:]
    
    tweet = TreebankWordDetokenizer().detokenize(wordsnew)
    tweet = tweet.replace("-","").replace("$","")
    return tweet

In [ ]:
def get_data(path):
    filepath_com = os.getcwd() + '/price/test_sample/'
    company = os.listdir(filepath_com)

    total_list = []
    for i in range(len(company[:-1])):   #len(company[:-1])
        filepath_date = os.getcwd() + '/tweet/preprocessed/' + str(company[i][:-4])
        X = pd.read_csv(os.getcwd()+'/price/' + path + str(company[i]))
        date = os.listdir(filepath_date)
        
        for j in range(len(date)): #len(date)-1
            if date[j] in X['date'].values.tolist() and date[j][:-4] is not '.ipynb_checkpo':
                #print(date[j])
                list_oneday = []
                list_oneday_score = []
                list = preprocessing(company[i][:-4], date[j])
                for k in range(len(list)):
                    sentence = Sentence(TreebankWordDetokenizer().detokenize(list[k]))
                    #print(sentence)
                    #print(cleaner(str(sentence)))
                    classifier.predict(sentence)
                    
                    list_oneday_score.append(sentence.labels[0].score)
                    oneday = (embed(list[k]))
                    
                    if oneday.shape[0] >= MAX_LENGTH:
                        list_oneday.append(oneday[0:MAX_LENGTH])
                    else:
                        one_as_vector = tf.reshape(oneday,[-1])
                        zero_paddings = tf.zeros([MAX_LENGTH*EMBEDDING_DIMENSION]-tf.shape(one_as_vector),dtype=oneday.dtype)
                        oneday_padded = tf.concat([one_as_vector, zero_paddings], 0)
                        results = tf.reshape(oneday_padded, [MAX_LENGTH, EMBEDDING_DIMENSION])
   
           
            
                        list_oneday.append(results)
                
                list_oneday_score = np.array(list_oneday_score)
                list_oneday_score = list_oneday_score/sum(list_oneday_score)
                res = list_oneday * list_oneday_score[:, np.newaxis, np.newaxis]
                #list_oneday = list_oneday*list_oneday_score
                list_oneday = tf.convert_to_tensor(res)
                #print(list_oneday)
                
               
                list_oneday = np.mean(list_oneday,axis=0)
                list_oneday = tf.convert_to_tensor(list_oneday)
                
                total_list.append(list_oneday)
        #print(len(total_list))
    total_list = tf.convert_to_tensor(total_list)
    #gru = tf.keras.layers.GRU(64)
    #total_list = (gru(total_list))
    print(total_list.shape)
    #np.save('embeddings.npy', embed(list[0]))
    #all_embeddings = np.load('embeddings.npy')
    #print(torch.FloatTensor(np.asarray(all_embeddings)))
    return total_list

In [ ]:
training_data = get_data('train_sample/')
validation_data = get_data('val_sample/')
testing_data = get_data('test_sample/')

In [ ]:
xtrain_input=[]
for i in range(len(training_data)-WINDOWS):
  xtrain_input.append(training_data[i:i+WINDOWS,:])
xtrain_input=np.array(xtrain_input)
print(xtrain_input.shape)
xtrain_input = xtrain_input.reshape(15222, 5, -1)

xval_input=[]
for i in range(len(validation_data)-WINDOWS):
  xval_input.append(validation_data[i:i+WINDOWS,:])
xval_input=np.array(xval_input)
print(xval_input.shape)
xval_input = xval_input.reshape(1654, 5, -1)

xtest_input=[]
for i in range(len(testing_data)-WINDOWS):
  xtest_input.append(testing_data[i:i+WINDOWS,:])
xtest_input=np.array(xtest_input)
print(xtest_input.shape)
xtest_input = xtest_input.reshape(2513, 5, -1)

In [ ]:
optim = keras.optimizers.RMSprop(
    learning_rate=3e-1,
    rho=0.9,
    momentum=0.1,
    epsilon=1e-07,
    centered=False,
    name="RMSprop",
)
print(xtrain_input.shape[1:])

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
    #return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mcc_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

In [ ]:
def create_model(input_shape):
    inputs = Input(shape=input_shape)
    x = TCN(nb_filters=16, return_sequences=True)(inputs)
  #  x = Bidirectional(GRU(64, input_shape=input_shape, return_sequences=True))(inputs)
  #  x = Dropout(0.3)(x)
    x = Dense(16,kernel_initializer="uniform",activation='relu')(x)
    for i in range(3):
        x1 = TCN(nb_filters=16, return_sequences=True)(inputs)
        x1 = Dense(16,kernel_initializer="uniform",activation='relu')(x1)
        x = concatenate([x,x1],axis=-1)

  
        
 #   x = Bidirectional(GRU(32, return_sequences=True))(x)
 #   x = Dropout(0.3)(x)
  #  x = Bidirectional(GRU(16, return_sequences=True))(x)
  #  x = Dropout(0.3)(x)
  #  x = SeqSelfAttention(attention_activation='relu')(x)
    
    #x = Bidirectional(GRU(16, return_sequences=False))(inputs)
   # x = Dropout(0.3)(x)
   
  #  x = Dense(32,kernel_initializer="uniform",activation='relu')(inputs)
   # x = Dense(8,kernel_initializer="uniform",activation='relu')(x)
    x = Dense(1,kernel_initializer="uniform",activation='relu')(x)
    
    model = Model(inputs=inputs, outputs = x)
    
    
    return model

In [ ]:
def custom_loss(y_true, y_pred):
 
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    bce_loss = bce(y_true, y_pred)
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = 0.5*bce_loss + 0.5*mse_loss
    
    return loss

In [ ]:
model = create_model(input_shape=xtrain_input.shape[1:])
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy', f1_m, mcc_m])

In [ ]:
my_callbacks = [
    # keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='modellstm.{epoch:02d}.h5'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]

history=model.fit(
    xtrain_input,
    y,
    batch_size=BATCH_SIZE,
    epochs=50,
    validation_data=(xval_input, y_val),
    verbose=1,
    callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

In [ ]:
test_acc = model.evaluate(xtest_input, y_test, batch_size = BATCH_SIZE)

In [ ]:
acc_hist = []
n = 1
for i in range(50):
  k = str(n).zfill(2)
  n += 1
  name = 'modellstm.' + k +'.h5'
  model11 = load_model(name, custom_objects={'SeqSelfAttention':SeqSelfAttention.get_custom_objects()['SeqSelfAttention'],'TCN':TCN,'f1_m':f1_m,'mcc_m':mcc_m})
  model11.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy', f1_m, mcc_m])
  loss,acc,f1,mcc = model11.evaluate(xtest_input,y_test,batch_size=BATCH_SIZE)
  acc_hist.append(acc)
  print(acc)
    
print(max(acc_hist))
print(acc_hist.index(max(acc_hist))+1)

In [ ]:
name = 'modellstm.26.h5'
model11 = load_model(name, custom_objects={'SeqSelfAttention':SeqSelfAttention.get_custom_objects()['SeqSelfAttention'],'TCN':TCN,'f1_m':f1_m,'mcc_m':mcc_m})
model11.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy', f1_m, mcc_m])
loss,acc,f1,mcc = model11.evaluate(xtest_input,y_test,batch_size=BATCH_SIZE)
print(acc)
print(f1)
print(mcc)

In [ ]:
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[0].output])
layer_output = get_3rd_layer_output(xtest_input)[0]
layer_output

In [ ]:
layer_output.shape